In this tutorial, we will show four ways to use generator:
1. Use directly by setting up a ``ModelClient``.
2. Customize prompt template using ``jinjia2``.
3. Try different models.
4. Use ``acall`` to do mutiple asynchronous calls for speed up.

In default, the generator uses a default prompt template. It has these varaibles:

LIGHTRAG_DEFAULT_PROMPT_ARGS = [
    "task_desc_str",
    "output_format_str",
    "tools_str",
    "examples_str",
    "chat_history_str",
    "context_str",
    "steps_str",
    "input_str",
    "output_str",
]

In [1]:
# first, let's set up the library log just in case, in default at INFO level
from utils.logger import get_logger
get_logger()

In [2]:
from adalflow.core import Generator
from adalflow.components.model_client import OpenAIClient
from adalflow.utils import setup_env # ensure you have .env with OPENAI_API_KEY

query = "What is the capital of France?"
model_kwargs = {
    "model": "gpt-3.5-turbo"
}
generator = Generator(model_client=OpenAIClient(), model_kwargs=model_kwargs)
prompt_kwargs = {
    "input_str": query,
}
# run the generator
output = generator(prompt_kwargs=prompt_kwargs)
print(output)

2024-06-09 22:06:49 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['chat_history_str', 'task_desc_str', 'steps_str', 'examples_str', 'tools_str', 'context_str', 'output_str', 'output_format_str', 'input_str']
2024-06-09 22:06:49 - INFO - [generator.py:194:call] - prompt_kwargs: {'input_str': 'What is the capital of France?'}
2024-06-09 22:06:49 - INFO - [generator.py:195:call] - model_kwargs: {}
2024-06-09 22:06:49 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<Inputs>\nWhat is the capital of France?\n</Inputs>'}]}
2024-06-09 22:06:49 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-09 22:06:49 - INFO - [generator.py:203:call] - output: GeneratorOutput(data='The capital of France is Paris.', error=None, raw_response='The capital of France is Paris.')
GeneratorOutput(data='The capital of France is Paris.',

The logging clearly shows us what we sent to OpenAI.

In [3]:
# lets see the prompt, it is quite minimal
generator.print_prompt(**prompt_kwargs)

Prompt:

<Inputs>
What is the capital of France?
</Inputs>



Writing your template is easy. Let us use our own template. Let's say, we want to set up our AI with a sense of humor.

In [7]:
template = """<SYS> Your are an assistant with a great sense of humor.</SYS> User: {{input_str}}. You:"""

generator2 = Generator(model_client=OpenAIClient(), model_kwargs=model_kwargs, template=template)
response = generator2(prompt_kwargs=prompt_kwargs)
print(response)

2024-06-09 22:09:43 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']
2024-06-09 22:09:43 - INFO - [generator.py:194:call] - prompt_kwargs: {'input_str': 'What is the capital of France?'}
2024-06-09 22:09:43 - INFO - [generator.py:195:call] - model_kwargs: {}


2024-06-09 22:09:43 - INFO - [openai_client.py:122:call] - api_kwargs: {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': '<SYS> Your are an assistant with a great sense of humor.</SYS> User: What is the capital of France?. You:'}]}
2024-06-09 22:09:44 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-09 22:09:44 - INFO - [generator.py:203:call] - output: GeneratorOutput(data='The capital of France is Paris. It\'s the city of love, pastries, and baguettes, so pack your beret and get ready to say "ooh la la!"', error=None, raw_response='The capital of France is Paris. It\'s the city of love, pastries, and baguettes, so pack your beret and get ready to say "ooh la la!"')
GeneratorOutput(data='The capital of France is Paris. It\'s the city of love, pastries, and baguettes, so pack your beret and get ready to say "ooh la la!"', error=None, raw_response='The capital of France is Paris.

In [8]:
# Let us use llama3 from groq
from lightrag.components.model_client import GroqAPIClient

groq_model_kwargs = {"model": "llama3-8b-8192"}
generator3 = Generator(model_client=GroqAPIClient(), model_kwargs=groq_model_kwargs, template=template)

response = generator3(prompt_kwargs=prompt_kwargs)
print(response)

2024-06-09 22:12:28 - INFO - [prompt_builder.py:82:__init__] - Prompt has variables: ['input_str']
2024-06-09 22:12:28 - INFO - [generator.py:194:call] - prompt_kwargs: {'input_str': 'What is the capital of France?'}
2024-06-09 22:12:28 - INFO - [generator.py:195:call] - model_kwargs: {}
2024-06-09 22:12:29 - INFO - [_client.py:1026:_send_single_request] - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2024-06-09 22:12:29 - INFO - [generator.py:203:call] - output: GeneratorOutput(data='Bonjour! The capital of France is indeed Paris! But did you know that Paris is also the city of love, famous for its Eiffel Tower, Louvre Museum, and, of course, croissants...', error=None, raw_response='Bonjour! The capital of France is indeed Paris! But did you know that Paris is also the city of love, famous for its Eiffel Tower, Louvre Museum, and, of course, croissants...')
GeneratorOutput(data='Bonjour! The capital of France is indeed Paris! But did you know th

In [11]:
# Lets do 10 async calls at once, lets use GroqAPIClient
import nest_asyncio # import asyncio, use nest_asyncio.apply() if you are in jupyter notebook
import asyncio
nest_asyncio.apply()

import time
from typing import List

async def make_async_calls(queries: List[str]):
    calls = [generator3.acall(prompt_kwargs={"input_str": query}) for query in queries]
    responses = await asyncio.gather(*calls)
    return responses

queries = [query] * 10
start = time.time()
responses = asyncio.run(make_async_calls(queries))
print(f"Time taken for 10 async calls: {time.time() - start}")
print(responses)



2024-06-09 22:20:08 - INFO - [generator.py:217:acall] - prompt_kwargs: {'input_str': 'What is the capital of France?'}
2024-06-09 22:20:08 - INFO - [generator.py:218:acall] - model_kwargs: {}
2024-06-09 22:20:08 - INFO - [generator.py:217:acall] - prompt_kwargs: {'input_str': 'What is the capital of France?'}
2024-06-09 22:20:08 - INFO - [generator.py:218:acall] - model_kwargs: {}
2024-06-09 22:20:08 - INFO - [generator.py:217:acall] - prompt_kwargs: {'input_str': 'What is the capital of France?'}
2024-06-09 22:20:08 - INFO - [generator.py:218:acall] - model_kwargs: {}
2024-06-09 22:20:08 - INFO - [generator.py:217:acall] - prompt_kwargs: {'input_str': 'What is the capital of France?'}
2024-06-09 22:20:08 - INFO - [generator.py:218:acall] - model_kwargs: {}
2024-06-09 22:20:08 - INFO - [generator.py:217:acall] - prompt_kwargs: {'input_str': 'What is the capital of France?'}
2024-06-09 22:20:08 - INFO - [generator.py:218:acall] - model_kwargs: {}
2024-06-09 22:20:08 - INFO - [generator.